# time match 가 끝난 raw를 edf로 export

In [1]:
import os
import mne
import pandas as pd
import EDFlib

In [2]:
path_subjects_edf = 'D:\\USC\\Sleep dataset\\Samsung_data\\NEW_EDF'
subjects_edf = os.listdir(path_subjects_edf)
subjects_edf.sort()
print(subjects_edf)

meas_time_df = pd.read_csv('D:\\USC\\code_mine\\Exploration_for_VD_dataset\\meas_time_edf_hypno.csv', index_col=0)
meas_time_df = meas_time_df.loc['LE004':'LE023']
meas_time_df

['LE004-1_edf', 'LE010-3_edf', 'LE011-1_edf', 'LE014-3_edf', 'LE019-1_edf', 'LE020-3_edf', 'LE021-2_edf', 'LE023-3_edf']


,hypno_start,hypno_end,edf_start,edf_duration,edf_end,start_mismatch,end_mismatch
LE004,0:31:00,5:52:00,0:17:47,5:34:26,5:52:13,0,0
LE010,0:24:30,6:36:30,0:16:31,6:20:54,6:37:25,0,0
LE011,0:13:00,6:39:00,0:10:39,6:36:12,6:46:51,0,0
LE014,0:24:30,6:08:00,0:18:44,5:49:56,6:08:40,0,0
LE019,0:32:00,6:18:30,0:12:15,6:07:08,6:19:23,0,0
LE020,0:24:30,6:13:00,0:19:18,5:54:02,6:13:20,0,0
LE021,0:35:00,6:08:00,0:32:14,5:36:08,6:08:22,0,0
LE023,0:33:30,5:45:00,0:32:55,5:13:18,5:46:13,0,0


In [3]:
now_edf = os.path.join(path_subjects_edf, subjects_edf[0], 'Traces.edf')
raw = mne.io.read_raw_edf(now_edf, preload=True, verbose=False)

[]


In [4]:
# Channel Select
eogs = ['SO', 'IO']
eegs = ['C3', 'C4']
pick_chs = eogs+eegs
raw.pick_channels(pick_chs)
chs = raw.ch_names
print("-- Available Channels: {}".format(chs))

-- Available Channels: ['SO', 'C3', 'C4', 'IO']


In [7]:
subject = subjects_edf[0]
subject

'LE004-1_edf'

In [5]:
import datetime
def str_time_to_seconds(time_str):
    time = time_str.split(':')
    seconds = int(time[0])*3600 + int(time[1])*60 + int(time[2])
    
    return seconds

def seconds_to_str_time(seconds):
    time = datetime.timedelta(seconds=seconds)
    return time

In [8]:
# start, end time matching
# -- get start, end times
hypno_start = meas_time_df.loc[subject[0:5], 'hypno_start']
hypno_end = meas_time_df.loc[subject[0:5], 'hypno_end']
edf_start = meas_time_df.loc[subject[0:5], 'edf_start']
edf_end = meas_time_df.loc[subject[0:5], 'edf_end']

hypno_start_sec = str_time_to_seconds(hypno_start)
hypno_end_sec = str_time_to_seconds(hypno_end)
edf_start_sec = str_time_to_seconds(edf_start)

new_edf_start_sec = (hypno_start_sec - 30) - edf_start_sec
new_edf_end_sec = hypno_end_sec - edf_start_sec

print(f"-- Edf time before matching: {edf_start}  {edf_end}")
print("-- Edf time after mathcing: {}  {}".format(
    str(seconds_to_str_time(edf_start_sec+new_edf_start_sec)),
    str(seconds_to_str_time(edf_start_sec+new_edf_end_sec))
))
print(f"-- ground_truth time: {hypno_start}  {hypno_end}")

-- Edf time before matching: 0:17:47  5:52:13
-- Edf time after mathcing: 0:30:30  5:52:00
-- ground_truth time: 0:31:00  5:52:00


In [9]:
raw.crop(tmin= new_edf_start_sec, tmax=new_edf_end_sec)

<RawEDF | Traces.edf, 4 x 9645001 (19290.0 s), ~294.4 MB, data loaded>

In [10]:
path_save = 'D:\\USC\\Sleep dataset\\Samsung_data\\NEW_EDF_matched'
save_as = os.path.join(path_save, subject[0:5]+'.edf')
mne.export.export_raw(save_as, raw, fmt='edf')

C:\Users\CYS\AppData\Local\Temp\ipykernel_9536\3149837832.py:3: RuntimeWarning: EDF format requires equal-length data blocks, so 0.998 seconds of zeros were appended to all channels when writing the final block.
  mne.export.export_raw(save_as, raw, fmt='edf')


In [12]:
f_name = 'D:\\USC\\Sleep dataset\\Samsung_data\\NEW_EDF_matched\\LE004.edf'
raw = mne.io.read_raw_edf(f_name, preload=True)

Extracting EDF parameters from D:\USC\Sleep dataset\Samsung_data\NEW_EDF_matched\LE004.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9645499  =      0.000 ... 19290.998 secs...
[]


In [13]:
raw

<RawEDF | LE004.edf, 4 x 9645500 (19291.0 s), ~294.4 MB, data loaded>